## Problem Statement for Potato Leaf Disease Prediction


Farmers who grow potatoes suffer from serious financial standpoint losses each year which cause several diseases that affect potato plants. The diseases Early Blight and Late Blight are the most frequent. Early blight is caused by fungus and late blight is caused by specific micro-organisms and if farmers detect this disease early and apply appropriate treatment then it can save a lot of waste and prevent economical loss. The treatments for early blight and late blight are a little different so it’s important that you accurately identify what kind of disease is there in that potato plant. Behind the scene, i am going to use Convolutional Neural Network – Deep Learning to diagnose plant diseases.

In [10]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [11]:
!kaggle datasets download -d muhammadardiputra/potato-leaf-disease-dataset

 20% 5.00M/25.2M [00:00<00:00, 35.6MB/s]
100% 25.2M/25.2M [00:00<00:00, 126MB/s] 


In [12]:
import zipfile
zip_ref = zipfile.ZipFile('/content/potato-leaf-disease-dataset.zip', 'r')
zip_ref.extractall('/content/Dataset')
zip_ref.close()

In [62]:
# import the libraries as shown below
import tensorflow as tf

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50

#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

from keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

In [14]:
IMAGE_SIZE = [224,224]

train_path = '/content/Dataset/Potato/Train'
valid_path = '/content/Dataset/Potato/Test'

In [15]:
vgg19 = tf.keras.applications.ResNet101V2(
    include_top=False,
    weights="imagenet",
    input_shape=IMAGE_SIZE + [3],
    )

171317808/171317808 [==============================] - 1s 0us/step


In [16]:
for layer in vgg19.layers:
  layer.trainable = False

In [17]:
folders = glob('/content/Dataset/Potato/Train/*')
folders

['/content/Dataset/Potato/Train/Potato___Early_blight',
 '/content/Dataset/Potato/Train/Potato___healthy',
 '/content/Dataset/Potato/Train/Potato___Late_blight']

In [18]:
x = Flatten()(vgg19.output)

In [19]:
prediction = Dense(len(folders), activation='softmax')(x)

# creating object

model = Model(inputs = vgg19.input, outputs =  prediction)

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [21]:
from keras.backend import categorical_crossentropy
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [22]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [23]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/Dataset/Potato/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',)

Found 900 images belonging to 3 classes.


In [24]:
test_set = test_datagen.flow_from_directory('/content/Dataset/Potato/Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 300 images belonging to 3 classes.


In [34]:
from keras import callbacks
checkpoint = ModelCheckpoint(filepath='/content/ResNet.h5', 
                             monitor='val_loss',
                             verbose=1, 
                             save_best_only=True,
                             mode='min')

earlystopping = [checkpoint]

In [36]:
r = model.fit(training_set,  
          validation_data=test_set, 
          epochs=20, 
          steps_per_epoch=len(training_set), 
          validation_steps=len(test_set), callbacks = earlystopping)

Epoch 1/20
29/29 [==============================] - ETA: 0s - loss: 0.0200 - accuracy: 0.9967
Epoch 1: val_loss improved from inf to 1.09466, saving model to /content/ResNet.h5
29/29 [==============================] - 14s 496ms/step - loss: 0.0200 - accuracy: 0.9967 - val_loss: 1.0947 - val_accuracy: 0.9200
Epoch 2/20
29/29 [==============================] - ETA: 0s - loss: 0.0667 - accuracy: 0.9933
Epoch 2: val_loss improved from 1.09466 to 0.70222, saving model to /content/ResNet.h5
29/29 [==============================] - 14s 466ms/step - loss: 0.0667 - accuracy: 0.9933 - val_loss: 0.7022 - val_accuracy: 0.9500
Epoch 3/20
29/29 [==============================] - ETA: 0s - loss: 0.0291 - accuracy: 0.9956
Epoch 3: val_loss did not improve from 0.70222
29/29 [==============================] - 13s 450ms/step - loss: 0.0291 - accuracy: 0.9956 - val_loss: 1.3067 - val_accuracy: 0.9133
Epoch 4/20
29/29 [==============================] - ETA: 0s - loss: 0.1244 - accuracy: 0.9878
Epoch 4: va

In [1]:
plt.plot(r.history['loss'],label = 'train loss')
plt.plot(r.history['val_loss'],label = 'val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

plt.plot(r.history['accuracy'],label = 'train accuracy')
plt.plot(r.history['val_accuracy'],label = 'val accuracy')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

NameError: name 'plt' is not defined

In [38]:
y_pred = model.predict(test_set)
y_pred

10/10 [==============================] - 3s 151ms/step


array([[0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.19688562e-15, 1.00000000e+00],
       [9.99952078e-01, 4.78643306e-05, 0.00000000e+00],
       [0.00000000e+00, 1.60390879e-13, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00, 2.83737996e-15],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [1.94053340e-32, 1.98173336e-14, 1.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.00000000e+00, 7.92080151e-11],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.67630387e-09, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.62037426e-24, 1.00000000e+00],
       [2.02621117e-18, 1.00000

In [39]:
import numpy as np
y_pred = np.argmax(y_pred, axis=0)
y_pred

array([8, 0, 2])

In [40]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [41]:
model=load_model('/content/ResNet.h5')

In [52]:
img=image.load_img('/content/Dataset/Potato/Test/Potato___Early_blight/0604174e-3018-4faa-9975-0be32d2c0789___RS_Early.B 7123.JPG',target_size=(224,224))

In [53]:
x=image.img_to_array(img)
x

array([[[175., 173., 186.],
        [170., 168., 181.],
        [166., 164., 177.],
        ...,
        [189., 189., 201.],
        [185., 185., 197.],
        [183., 183., 195.]],

       [[173., 171., 184.],
        [176., 174., 187.],
        [178., 176., 189.],
        ...,
        [192., 192., 204.],
        [188., 188., 200.],
        [185., 185., 197.]],

       [[174., 172., 185.],
        [183., 181., 194.],
        [191., 189., 202.],
        ...,
        [193., 193., 205.],
        [190., 190., 202.],
        [187., 187., 199.]],

       ...,

       [[215., 212., 223.],
        [118., 115., 126.],
        [141., 138., 149.],
        ...,
        [172., 169., 180.],
        [150., 147., 158.],
        [171., 168., 179.]],

       [[177., 174., 185.],
        [120., 117., 128.],
        [134., 131., 142.],
        ...,
        [190., 187., 198.],
        [147., 144., 155.],
        [149., 146., 157.]],

       [[139., 136., 147.],
        [166., 163., 174.],
        [122., 1

In [54]:
x.shape

(224, 224, 3)

In [55]:
x=x/255

In [56]:
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [57]:
model.predict(img_data)

1/1 [==============================] - 0s 52ms/step


array([[0., 1., 0.]], dtype=float32)

In [58]:
a=np.argmax(model.predict(img_data), axis=1)

1/1 [==============================] - 0s 36ms/step


In [61]:
a==1

array([ True])